In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge

In [5]:
data = pd.read_csv('Electricity BILL.csv')

X = data.drop('Electricity_Bill', axis=1)
y = data['Electricity_Bill']


categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(exclude=['object']).columns

one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)  
X_categorical_encoded = one_hot_encoder.fit_transform(X[categorical_features])

X_categorical_encoded_df = pd.DataFrame(X_categorical_encoded, columns=one_hot_encoder.get_feature_names_out(categorical_features))
X_numerical = X[numerical_features].reset_index(drop=True)
X_encoded = pd.concat([X_numerical, X_categorical_encoded_df], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
def evaluate_elasticnet(alpha_value, X_train, X_test, y_train, y_test):
    elasticnet_model = ElasticNet(alpha=alpha_value, random_state=42)
    
    elasticnet_model.fit(X_train, y_train)

    y_train_pred = elasticnet_model.predict(X_train)
    y_test_pred = elasticnet_model.predict(X_test)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_adjusted_r2 = 1 - (1 - train_r2) * (len(y_train) - 1) / (len(y_train) - X_train.shape[1] - 1)
    
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_adjusted_r2 = 1 - (1 - test_r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)
    
    print(f"=== ElasticNet with alpha={alpha_value} ===")
    print(f"Train MSE: {train_mse}")
    print(f"Test MSE: {test_mse}")
    print(f"Train RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")
    print(f"Train MAE: {train_mae}")
    print(f"Test MAE: {test_mae}")
    print(f"Train R2: {train_r2}")
    print(f"Test R2: {test_r2}")
    print(f"Train Adjusted R2: {train_adjusted_r2}")
    print(f"Test Adjusted R2: {test_adjusted_r2}")
    print("\n")
    
    return {
        'alpha': alpha_value,
        'train_mse': train_mse, 'test_mse': test_mse,
        'train_rmse': train_rmse, 'test_rmse': test_rmse,
        'train_mae': train_mae, 'test_mae': test_mae,
        'train_r2': train_r2, 'test_r2': test_r2,
        'train_adjusted_r2': train_adjusted_r2, 'test_adjusted_r2': test_adjusted_r2
    }


In [7]:

alpha_values = [0.1, 0.5, 1, 5, 10]

results = []

for alpha in alpha_values:
    result = evaluate_elasticnet(alpha, X_train, X_test, y_train, y_test)
    results.append(result)


=== ElasticNet with alpha=0.1 ===
Train MSE: 24190404.855616152
Test MSE: 24108417.937015377
Train RMSE: 4918.374208579106
Test RMSE: 4910.0323763714
Train MAE: 3976.0822914657842
Test MAE: 3796.386129678228
Train R2: 0.025389147880074514
Test R2: 0.007022754250851082
Train Adjusted R2: 0.006493631359382079
Test Adjusted R2: -0.0750058008327743


=== ElasticNet with alpha=0.5 ===
Train MSE: 24213520.011309635
Test MSE: 24063500.556266405
Train RMSE: 4920.723525185055
Test RMSE: 4905.456202665192
Train MAE: 3975.2409206961875
Test MAE: 3793.834544489474
Train R2: 0.02445785790283883
Test R2: 0.008872810821917154
Train Adjusted R2: 0.005544285760138767
Test Adjusted R2: -0.07300291350148957


=== ElasticNet with alpha=1 ===
Train MSE: 24255567.395840995
Test MSE: 24051287.377304178
Train RMSE: 4924.994151858557
Test RMSE: 4904.211188081543
Train MAE: 3976.290815429076
Test MAE: 3795.203184932364
Train R2: 0.022763804516295005
Test R2: 0.00937584709909789
Train Adjusted R2: 0.003817388481